In [2]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [3]:
import os

# Set the OpenAI API key globally
os.environ["OPENAI_API_KEY"] = "YOURKEY"
# Set Google API key and Search Engine ID directly in the notebook
os.environ["GOOGLE_API_KEY"] = "YOURKEYI"
os.environ["GOOGLE_SEARCH_ENGINE_ID"] = "YOURUD"


In [4]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:
    import os
    import time
    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")
    
    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": api_key, "cx": search_engine_id, "q": query, "num": num_results}

    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            return f"Error fetching {url}: {str(e)}"

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append({"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body})
        time.sleep(1)

    return enriched_results


In [5]:
def arxiv_search(query: str, max_results: int = 2) -> list:
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append({
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "published": paper.published.strftime("%Y-%m-%d"),
            "abstract": paper.summary,
            "pdf_url": paper.pdf_url,
        })

    return results


In [6]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define the tools
google_search_tool = FunctionTool(
    google_search,
    description="Search Google for information, returns results with a snippet and body content"
)

arxiv_search_tool = FunctionTool(
    arxiv_search,
    description="Search Arxiv for papers related to a given topic, including abstracts"
)
# Define agents

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
    Google Search Agent: Perform Research in Google to Gather Up-to-date infromations, insights, statstics, keywords, ideas, trends. you will ask for outup it depend to the task. also you can can override/bypass him if you find his particpation not necessary.
    Arvix Search Agent: This agents searches for informations, articles, litteratures to support the content production.  you will ask for outup it depend to the task. also you can can override/bypass him if you find his particpation not necessary.
    Content Creator Agent: This Agent has a Phd In English , Phd in Marketing, Phd in Writting. He will write any content output based on the data he will receive (litterature review, website content, articles, marketing content,, etc)

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)


google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Google for information and provide enriched content",
    system_message="You are a helpful assistant skilled in performing web searches and summarizing results."
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Arxiv for academic papers",
    system_message="You are a helpful assistant skilled in finding relevant academic papers."
)

content_creator_agent = AssistantAgent(
    name="Content_Creator",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Generate a structured literature review based on the data collected, Or any other type of Content",
    system_message="Your task is to synthesize data into a high-quality literature review with proper citations.End your response with 'TERMINATE'."
)



In [7]:
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_core import CancellationToken
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, google_search_agent, arxiv_search_agent,content_creator_agent],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    termination_condition=termination,
)


In [8]:
task = "Write a literature review on no-code tools for the impact of AI on political Purposes"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Write a literature review on no-code tools for the impact of AI on political Purposes
---------- PlanningAgent ----------
To effectively address the task of writing a literature review on no-code tools and the impact of AI on political purposes, I will begin by breaking down the process into manageable subtasks.

1. Google Search Agent: Research and gather up-to-date information on no-code tools and their applications in political contexts. Look for statistics, keywords, trends, and related insights.

2. Arvix Search Agent: Search for academic articles, papers, and literature that discuss the relationship between no-code tools, AI, and political purposes. Focus on empirical studies, theoretical frameworks, and case studies.

3. Content Creator Agent: Based on the information provided by the Google Search Agent and the Arvix Search Agent, write a comprehensive literature review on no-code tools for the impact of AI on political purposes.

Once these tasks are 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a literature review on no-code tools for the impact of AI on political Purposes', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=282, completion_tokens=292), content="To effectively address the task of writing a literature review on no-code tools and the impact of AI on political purposes, I will begin by breaking down the process into manageable subtasks.\n\n1. Google Search Agent: Research and gather up-to-date information on no-code tools and their applications in political contexts. Look for statistics, keywords, trends, and related insights.\n\n2. Arvix Search Agent: Search for academic articles, papers, and literature that discuss the relationship between no-code tools, AI, and political purposes. Focus on empirical studies, theoretical frameworks, and case studies.\n\n3. Content Creator Agent: Based on the information provided by the Google Search A

In [ ]:
from autogen_core.tools import FunctionTool
